In [1]:
import torch
import torch.nn as nn
import numpy as np

from torch.utils.data import Dataset, DataLoader

In [2]:
""" Setting up a simple neural network model"""
class NeuralNetwork(nn.Module):
    def __init__(self, n_inputs, n_hidden, n_outputs):
        super(NeuralNetwork, self).__init__()
        self.hidden = nn.Linear(n_inputs, n_hidden)
        self.output = nn.Linear(n_hidden, n_outputs)
        self.activation = nn.ReLU()

    def forward(self, x):
        return self.output(self.activation(self.hidden(x)))

In [3]:
"""Setting up a simple neural network model with dropout"""
class NeuralNetworkDropout(nn.Module):
    def __init__(self, n_inputs, n_hidden, n_outputs, p):
        super(NeuralNetworkDropout, self).__init__()
        self.p = p
        self.network = nn.Sequential(nn.Linear(n_inputs, n_hidden),
                                     nn.ReLU(),
                                     nn.Linear(n_hidden, n_hidden*2),
                                     nn.ReLU(),
                                     nn.Dropout(self.p),
                                     nn.Linear(n_hidden*2, n_outputs))
    
    def forward(self, x):
        return self.network(x)

In [ ]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, input_size, hidden_dim, vocab_size, target_size, num_layers):
        super().__init__()
        self.target_size = target_size
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.output = nn.Linear(hidden_dim, target_size)

    def forward(self, x, prev_state):
        batch_size = x.size(0)
        embeds = self.word_embeddings(sentence)
        lstm_out, state = self.lstm(embeds, prev_state)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        out = self.output(lstm_out)
        out = out.view(batch_size, -1, self.target_size)
        out = out[:, -1]

        # return one batch of output word scores and the hidden state
        return out, state

In [4]:
model = NeuralNetworkDropout(3, 4, 3, 0.1)

In [5]:
x = torch.randn(100, 3)

y = x * np.sin(x) + 0.01

In [6]:
class ExampleDataset(Dataset):
    def __init__(self, x, y):
        super(ExampleDataset, self).__init__()
        self.x = x
        self.y = y
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.x.shape[0]

In [7]:
dataset = ExampleDataset(x, y)

In [8]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [9]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

criterion = nn.MSELoss()

epochs = 100
for epoch in range(epochs):
    for i, (x, y) in enumerate(dataloader):
        y_hat = model(x)
        loss = criterion(y_hat, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [10]:
with torch.no_grad():
    model.eval()